In [5]:

from os import listdir
from os.path import isfile, join

import pandas as pd
import plotly

#from data_filters import *

STATE_DATA_PATH = "../data/hmda_state_data/"
OUTPUT_PATH = "../output/"

sf_1st_purch = {
    "construction_method":["1"],
    "total_units":["1", "2", "3", "4"],
    "loan_purpose":["1"],
    "open-end_line_of_credit":["2"],
    "reverse_mortgage":["2"],
    "lien_status":["1"]
}

def filter_hmda_df(data_df, filter_dict):
    """
    filter dict is filter name: value list and is used to limit the dataframe return
    filters use the pandas .isin() function to accomodate multiple values 
    
    """
    #filter data by dictionary
    for key in filter_dict.keys():
        data_df = data_df[data_df[key].isin(filter_dict[key])].copy()
    return data_df

#FIXME: state level aggregation needs to account for mismatches between state letter and state fips
def aggregate_by_geo(geo_data, geo_name_code, geo_type="state_code"):
    """
    geo types are: state_code, county_code, census_tract, derived_msa_md
    geo_data_file is a pandas dataframe
    geo_name_code: the geographic identifier of the geo_data passed. for example FL for florida state
    """
    agg_data = {
            geo_type:geo_name_code,
            "sf_app_count":filter_hmda_df(geo_data, sf_1st_purch).action_taken.count(), 
            "sf_orig_count":filter_hmda_df(filter_hmda_df(geo_data, sf_1st_purch), 
                                          {"action_taken":["1"]}).action_taken.count(), 
            "sf_app_val":round(filter_hmda_df(geo_data, sf_1st_purch).loan_amount.astype(float).sum())/1000, 
            "sf_orig_val":round(filter_hmda_df(filter_hmda_df(geo_data, sf_1st_purch), 
                                             {"action_taken":["1"]}).loan_amount.astype(float).sum())/1000,
            "sf_orig_pct": round(filter_hmda_df(filter_hmda_df(geo_data, sf_1st_purch), 
                                          {"action_taken":["1"]}).action_taken.count() / 
                                 filter_hmda_df(geo_data, sf_1st_purch).action_taken.count()*100,2), 
            "sf_avg_app_val":round(filter_hmda_df(geo_data, sf_1st_purch).loan_amount.astype(float).mean(),0)/1000, 
            "sf_avg_orig_val":round(filter_hmda_df(filter_hmda_df(geo_data, sf_1st_purch), 
                                          {"action_taken":["1"]}).loan_amount.astype(float).mean(),0)/1000,
            "sf_med_app_val":round(filter_hmda_df(geo_data, sf_1st_purch).loan_amount.astype(float).median(),0)/1000, 
            "sf_med_orig_val":round(filter_hmda_df(filter_hmda_df(geo_data, sf_1st_purch), 
                                          {"action_taken":["1"]}).loan_amount.astype(float).median(),0)/1000,

    }
    return agg_data

In [22]:
#get all state files
state_files = [f for f in listdir(STATE_DATA_PATH) if isfile(join(STATE_DATA_PATH, f))]
state_agg_data = []
for state in state_files:
    print("processing:", state)
    state_df = pd.read_csv(STATE_DATA_PATH + state, low_memory=False)
    state_agg_data.append(aggregate_by_geo(geo_data=state_df, geo_name_code=state[:-4], 
                                           geo_type="state_code"))
    
#print(state_agg_data)
state_agg_df = pd.DataFrame(state_agg_data)
state_agg_df.to_csv(OUTPUT_PATH + "state_aggregates.txt", index=False)
state_agg_df


processing: AK.csv
processing: AL.csv
processing: AR.csv
processing: AS.csv
processing: AZ.csv


c:\python37\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning:

invalid value encountered in long_scalars



processing: CA.csv
processing: CO.csv
processing: CT.csv
processing: DC.csv
processing: DE.csv
processing: FL.csv
processing: GA.csv
processing: GU.csv
processing: HI.csv
processing: IA.csv
processing: ID.csv
processing: IL.csv
processing: IN.csv
processing: KS.csv
processing: KY.csv
processing: LA.csv
processing: MA.csv
processing: MD.csv
processing: ME.csv
processing: MI.csv
processing: MN.csv
processing: MO.csv
processing: MP.csv
processing: MS.csv
processing: MT.csv
processing: NC.csv
processing: ND.csv
processing: NE.csv
processing: NH.csv
processing: NJ.csv
processing: NM.csv
processing: NV.csv
processing: NY.csv
processing: OH.csv
processing: OK.csv
processing: OR.csv
processing: PA.csv
processing: PR.csv
processing: RI.csv
processing: SC.csv
processing: SD.csv
processing: TN.csv
processing: TX.csv
processing: UT.csv
processing: VA.csv
processing: VI.csv
processing: VT.csv
processing: WA.csv
processing: WI.csv
processing: WV.csv
processing: WY.csv


,state_code,sf_app_count,sf_orig_count,sf_app_val,sf_orig_val,sf_orig_pct,sf_avg_app_val,sf_avg_orig_val,sf_med_app_val,sf_med_orig_val
0,AK,15256,8572,4306410.0,2441080.0,56.19,282.276,284.774,265.0,265.0
1,AL,94882,56480,17907020.0,10881830.0,59.53,188.729,192.667,165.0,165.0
2,AR,59990,33967,9893770.0,5691465.0,56.62,164.924,167.559,145.0,145.0
3,AS,0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,AZ,219700,121081,55728950.0,31078325.0,55.11,253.659,256.674,225.0,225.0
5,CA,598707,344877,295399075.0,173866785.0,57.60,493.395,504.141,405.0,405.0
6,CO,174868,104113,60958180.0,36068085.0,59.54,348.595,346.432,315.0,315.0
7,CT,62267,37942,17644445.0,10931720.0,60.93,283.368,288.117,225.0,225.0
8,DC,14422,8144,7591470.0,4409650.0,56.47,526.381,541.460,455.0,465.0
9,DE,25945,14267,6515995.0,3644045.0,54.99,251.146,255.418,225.0,235.0


In [23]:
county_agg_data = []
for state in state_files:
    state_df = pd.read_csv(STATE_DATA_PATH + state, dtype=object, low_memory=False)
    state_df.county_code.fillna("Not Reported", inplace=True)
    state_df.county_code = state_df.county_code.apply(lambda x: str(x).replace(".0", ""))
    print("processing:", state)
    for county_fips in list(state_df.county_code.unique()):
        
        print("processing:", county_fips)
        county_df = state_df[state_df.county_code==county_fips].copy()
        county_agg_data.append(aggregate_by_geo(geo_data=county_df, geo_name_code=county_fips, 
                                           geo_type="county_code"))
county_agg_df = pd.DataFrame(county_agg_data)
county_agg_df.to_csv(OUTPUT_PATH + "county_aggregates.txt", index=False)
county_agg_df

processing: AK.csv
processing: 2170
processing: 2020
processing: 2188
processing: 2090
processing: 2122
processing: 2110
processing: 2195
processing: 2220
processing: 2261
processing: 2150
processing: 2185
processing: 2130
processing: 2100
processing: 2050
processing: 2105
processing: 2240
processing: 2016
processing: 2198
processing: 2180
processing: 2230
processing: 2070
processing: 2275
processing: 2290
processing: Not Reported
processing: 2164
processing: 2282
processing: 2068
processing: 2060
processing: 5035
processing: 5005
processing: 2013
processing: 2158
processing: 5033
processing: 5143
processing: 5031
processing: 4025
processing: 48201


c:\python37\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning:

invalid value encountered in long_scalars



processing: AL.csv
processing: 1013
processing: 1097
processing: 1089
processing: 1061
processing: 1015
processing: 1101
processing: 1045
processing: 1017
processing: 1073
processing: 1125
processing: 1051
processing: 1069
processing: 1117
processing: 1003
processing: 1039
processing: 1001
processing: 1123
processing: 1067
processing: 1109
processing: 1053
processing: 1023
processing: 1047
processing: 1111
processing: 1105
processing: 1129
processing: 1085
processing: 1035
processing: 1115
processing: 1121
processing: 1029
processing: 1055
processing: 1027
processing: 1033
processing: 1093
processing: 1059
processing: 1103
processing: 1049
processing: 1079
processing: 1083
processing: 1081
processing: 1127
processing: 1095
processing: 1087
processing: 1031
processing: 1043
processing: 1077
processing: 1025
processing: 1075
processing: 1107
processing: 1113
processing: 1071
processing: 1009
processing: 1063
processing: 1021
processing: 1019
processing: 1005
processing: 1099
processing: 

processing: 6023
processing: 6003
processing: 6109
processing: 6057
processing: 6055
processing: 6015
processing: 6047
processing: 6007
processing: 6031
processing: 6101
processing: 6093
processing: 6027
processing: 6043
processing: 5115
processing: 6049
processing: 6103
processing: 6021
processing: 6035
processing: 6005
processing: 6011
processing: 6063
processing: 6105
processing: 6091
processing: 12015
processing: 13121
processing: 39099
processing: 39049
processing: 48257
processing: 48167
processing: 47149
processing: 17043
processing: 12081
processing: 53077
processing: 13063
processing: 12085
processing: 53063
processing: 34025
processing: 26087
processing: 12103
processing: 22103
processing: 37063
processing: 48209
processing: 34029
processing: 12011
processing: 17031
processing: 37157
processing: 13067
processing: 53011
processing: 47093
processing: 48491
processing: 53033
processing: 17089
processing: 47021
processing: 8035
processing: 13151
processing: 42115
processing: 8069

processing: 21085
processing: 16055
processing: 29189
processing: 45067
processing: 51137
processing: 8049
processing: 4027
processing: 32510
processing: 49035
processing: 8101
processing: 55131
processing: 55079
processing: 49011
processing: 55133
processing: 42017
processing: 9001
processing: 15003
processing: 21093
processing: 32023
processing: CO.csv
processing: 8013
processing: 8059
processing: 8035
processing: 8001
processing: 8041
processing: 8031
processing: 8083
processing: 8123
processing: 8007
processing: 8101
processing: 8119
processing: 8077
processing: 8051
processing: 8069
processing: 8039
processing: 8117
processing: 8005
processing: 8014
processing: 8065
processing: 8093
processing: 8113
processing: 8097
processing: 8067
processing: 8037
processing: 8107
processing: Not Reported
processing: 8029
processing: 8045
processing: 8043
processing: 8015
processing: 8047
processing: 8109
processing: 8023
processing: 8071
processing: 8125
processing: 8053
processing: 8075
proces

...

processing: 42009
processing: 42007
processing: 42059
processing: 42057
processing: 42085
processing: 42131
processing: 42111
processing: 42053
processing: 42065
processing: 42023
processing: 42123
processing: 24033
processing: 13071
processing: 47021
processing: 34025
processing: 18097
processing: 4013
processing: 13067
processing: 12085
processing: 6059
processing: 8029
processing: 26125
processing: 12055
processing: 12086
processing: 8123
processing: 8093
processing: 6107
processing: 8067
processing: 6037
processing: 48167
processing: 24005
processing: 48201
processing: 48427
processing: 41067
processing: 8059
processing: 12099
processing: 24510
processing: 6031
processing: 13097
processing: 6065
processing: 47129
processing: 39109
processing: 13089
processing: 13121
processing: 6071
processing: 41009
processing: 17031
processing: 41039
processing: 47093
processing: 26117
processing: 18067
processing: 48217
processing: 6001
processing: 8019
processing: 6073
processing: 18057
process

processing: 47077
processing: 47079
processing: 47109
processing: 47039
processing: 47047
processing: 47185
processing: 47097
processing: 47023
processing: 47117
processing: 47129
processing: 47071
processing: 47099
processing: 47089
processing: 47141
processing: 47143
processing: 47051
processing: 47029
processing: 47059
processing: 47159
processing: 47103
processing: 47003
processing: 47049
processing: 47173
processing: 47133
processing: 47025
processing: 47127
processing: 47169
processing: 47027
processing: 47057
processing: 47161
processing: 47015
processing: 47007
processing: 47181
processing: 47067
processing: 47135
processing: 13083
processing: 47137
processing: 47175
processing: 28057
processing: 1127
processing: 1043
processing: 27035
processing: 42101
processing: 12057
processing: 13067
processing: 48201
processing: 24003
processing: 6113
processing: 48439
processing: 4013
processing: 39151
processing: 6037
processing: 6071
processing: 41043
processing: 13151
processing: 6019

processing: 34029
processing: 53063
processing: 13013
processing: 6059
processing: 47035
processing: 6053
processing: 37001
processing: 13127
processing: 34013
processing: 4025
processing: 13223
processing: 37057
processing: 6097
processing: 41051
processing: 6025
processing: 18049
processing: 37171
processing: 27163
processing: 6085
processing: 8005
processing: 26137
processing: 13151
processing: 13121
processing: 4005
processing: 8045
processing: 22015
processing: 6107
processing: 47073
processing: 1043
processing: 41067
processing: 47081
processing: 4015
processing: 4019
processing: 12053
processing: 12099
processing: 8059
processing: 26099
processing: 8041
processing: 47001
processing: 6099
processing: 41039
processing: 37109
processing: 8001
processing: 12095
processing: 12071
processing: 22071
processing: 6083
processing: 6067
processing: 42107
processing: 8077
processing: 12073
processing: 27171
processing: 17043
processing: 26139
processing: 41005
processing: 8043
processing: 1

processing: 37159
processing: 12111
processing: 25027
processing: 37181
processing: 36069
processing: 11001
processing: 24035
processing: 37057
processing: 6081
processing: 53073
processing: 24013
processing: 54003
processing: 37091
processing: 24021
processing: 54077
processing: 37107
processing: 99999
processing: 42089
processing: 54079
processing: 24005
processing: 24027
processing: 24045
processing: 12081
processing: 42025
processing: 51014
processing: 47189
processing: 47163
processing: 37055
processing: 37089
processing: 54055
processing: 37073
processing: 21067
processing: 29095
processing: 36047
processing: 21195
processing: 21127
processing: 25017
processing: 29073
processing: 42103
processing: 40097
processing: 37169
processing: 23031
processing: VI.csv
processing: 78030
processing: 78010
processing: Not Reported
processing: VT.csv
processing: 50005
processing: 50027
processing: 50003
processing: 50025
processing: 50015
processing: 50017
processing: 50021
processing: 50011
pr

,county_code,sf_app_count,sf_orig_count,sf_app_val,sf_orig_val,sf_orig_pct,sf_avg_app_val,sf_avg_orig_val,sf_med_app_val,sf_med_orig_val
0,2170,3240,1853,866460.0,495575.0,57.19,267.426,267.445,245.0,255.0
1,2020,6805,4021,2109515.0,1245885.0,59.09,309.995,309.845,295.0,295.0
2,2188,25,11,5095.0,2435.0,44.00,203.800,221.364,175.0,205.0
3,2090,1703,912,419085.0,224970.0,53.55,246.086,246.678,235.0,245.0
4,2122,1336,719,314240.0,170785.0,53.82,235.210,237.531,225.0,225.0
...,...,...,...,...,...,...,...,...,...,...
6844,56017,71,41,12045.0,6445.0,57.75,169.648,157.195,165.0,155.0
6845,56031,148,55,25670.0,9665.0,37.16,173.446,175.727,165.0,165.0
6846,56027,14,10,1890.0,1240.0,71.43,135.000,124.000,155.0,130.0
6847,53045,0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [10]:
test_state = pd.read_csv(STATE_DATA_PATH+ state_files[0])
test_state.head()


c:\python37\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning:

Columns (22,23,24,26,27,28,29,30,31,32,33,38,44) have mixed types. Specify dtype option on import or set low_memory=False.



,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2018,7H6GLXDRUGQFU57RNE97,11260,AK,2170.0,2.170001e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,2274,18.12,97600,86,621,1786,23
1,2018,549300EHC6INE5XY6I44,11260,AK,2020.0,2.020003e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6375,49.93,97600,112,1739,2107,27
2,2018,549300EHC6INE5XY6I44,11260,AK,2020.0,2.020002e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5134,33.11,97600,106,1098,1559,38
3,2018,549300EHC6INE5XY6I44,11260,AK,2020.0,2.020002e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4999,34.09,97600,141,1477,1958,46
4,2018,549300EHC6INE5XY6I44,99999,AK,2188.0,2.188000e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3283,78.92,81200,113,431,885,32


In [ ]:

#discuss HMDA Datasets
#mixed data types, geographics are strings not int (important due to leading 0s)
# - distinctness of geographies without concatenation
#snapshot vs dynamic
# - LAR
# - TS
# - Panel
#   identifiers for external data, hierarchy, institution types
# - MSA MD description file
# what to do with each and in combination
# mapping between files on lei

#special features of HMDA: transaction dataset (includes non-originated), income, spatial analysis
# new features: manufactured housing breakout: chattle vs real property
# interest rate and rate spread (gives implied benchmark)
# units (bucketed)
# exemption: smaller depositories, may have outsized rural impact, shifts in baseline
# 
#the below lists of fields are critical to understand when working with HMDA data.
#analysis done without accounting for differences between loan or property types will yield
#questionable results, at best.
#additionally, analysts need to be aware of the action_taken field as HMDA contains applications that 
#did not result in originations as well as the originated covered loans.
property_filters = ['derived_dwelling_category','construction_method','occupancy_type', 
                    'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units',
                    'multifamily_affordable_units']
#there are more loan product features that can be used, but these are the basic ones to be aware of.
product_filters = ['loan_type', 'loan_purpose', 'lien_status', 'open-end_line_of_credit', 
                   'reverse_mortgage','business_or_commercial_purpose', 'loan_term',
                   'preapproval','submission_of_application','initially_payable_to_institution',
                   'derived_loan_product_type', 'conforming_loan_limit']


#differentiate by product types
# - single family, first lien, home purchase
#        - show breakout by loan type: conventional, FHA, RHS, VA
#note on exemption exclusion for types of analysis

# - single family, first lien, refinance
#        - show breakout by loan type: conventional, FHA, RHS, VA
# - single family, first lien, home improvement
#        - show breakout by lien status
#        - show breakout by loan type: conventional, FHA, RHS, VA
#       - 1st lien home improvement is odd, indicates clear ownership otherwise
# - manufactured housing
# - multifamily housing
# - MF affordable units NA vs 0

#create aggregates
# bank vs nonbank lending count and volume (percentage heatmap)
# manufactured housing by geography
#map aggregates
